In [1]:
import pandas as pd
import gc
from google.colab import drive
drive.mount('/content/drive')

data_root = '/content/drive/MyDrive/부트캠프/'
data_path = data_root + '11_파이널/'

Mounted at /content/drive


In [2]:
from imblearn.over_sampling import SMOTE

smote=SMOTE(sampling_strategy='minority')

In [2]:
df31 = pd.read_parquet(data_path + 'train/1.회원정보/201807_train_회원정보.parquet')
df32 = pd.read_parquet(data_path + 'train/2.신용정보/201807_train_신용정보.parquet')
df33 = pd.read_parquet(data_path + 'train/3.승인매출정보/201807_train_승인매출정보.parquet')
df34 = pd.read_parquet(data_path + 'train/4.청구입금정보/201807_train_청구정보.parquet')
df35 = pd.read_parquet(data_path + 'train/5.잔액정보/201807_train_잔액정보.parquet')
df36 = pd.read_parquet(data_path + 'train/6.채널정보/201807_train_채널정보.parquet')
df37 = pd.read_parquet(data_path + 'train/7.마케팅정보/201807_train_마케팅정보.parquet')
df38 = pd.read_parquet(data_path + 'train/8.성과정보/201807_train_성과정보.parquet')

In [5]:
df31 = pd.read_parquet(data_path + 'test/1.회원정보/201807_test_회원정보.parquet')
df32 = pd.read_parquet(data_path + 'test/2.신용정보/201807_test_신용정보.parquet')
df33 = pd.read_parquet(data_path + 'test/3.승인매출정보/201807_test_승인매출정보.parquet')
df34 = pd.read_parquet(data_path + 'test/4.청구입금정보/201807_test_청구정보.parquet')
df35 = pd.read_parquet(data_path + 'test/5.잔액정보/201807_test_잔액정보.parquet')
df36 = pd.read_parquet(data_path + 'test/6.채널정보/201807_test_채널정보.parquet')
df37 = pd.read_parquet(data_path + 'test/7.마케팅정보/201807_test_마케팅정보.parquet')
df38 = pd.read_parquet(data_path + 'test/8.성과정보/201807_test_성과정보.parquet')

In [3]:
train_df = df31.merge(df32, on=['기준년월', 'ID'], how='left')
del df32
del df31
gc.collect()
# train_df = train_df.merge(df33, on=['기준년월', 'ID'], how='left')
# del df33
# gc.collect()
train_df = train_df.merge(df34, on=['기준년월', 'ID'], how='left')
del df34
gc.collect()
train_df = train_df.merge(df35, on=['기준년월', 'ID'], how='left')
del df35
gc.collect()
train_df = train_df.merge(df36, on=['기준년월', 'ID'], how='left')
del df36
gc.collect()
train_df = train_df.merge(df37, on=['기준년월', 'ID'], how='left')
del df37
gc.collect()
train_df = train_df.merge(df38, on=['기준년월', 'ID'], how='left')
del df38
gc.collect()

6

In [4]:
train_df = train_df.merge(df33, on=['기준년월', 'ID'], how='left')
del df33
gc.collect()

9

In [6]:
test_df = df31.merge(df32, on=['기준년월', 'ID'], how='left')
del df32
del df31
gc.collect()
# train_df = train_df.merge(df33, on=['기준년월', 'ID'], how='left')
# del df33
# gc.collect()
test_df = test_df.merge(df34, on=['기준년월', 'ID'], how='left')
del df34
gc.collect()
test_df = test_df.merge(df35, on=['기준년월', 'ID'], how='left')
del df35
gc.collect()
test_df = test_df.merge(df36, on=['기준년월', 'ID'], how='left')
del df36
gc.collect()
test_df = test_df.merge(df37, on=['기준년월', 'ID'], how='left')
del df37
gc.collect()
test_df = test_df.merge(df38, on=['기준년월', 'ID'], how='left')
del df38
gc.collect()

0

In [7]:
test_df = test_df.merge(df33, on=['기준년월', 'ID'], how='left')
del df33
gc.collect()

6

In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400000 entries, 0 to 399999
Columns: 858 entries, 기준년월 to 이용금액대
dtypes: float64(61), int64(747), object(50)
memory usage: 2.6+ GB


In [6]:
train_df.to_parquet(data_path + 'processed/07_train_df.parquet')

In [8]:
test_df.to_parquet(data_path + 'processed/07_test_df.parquet')

In [13]:
train_df = pd.read_parquet(data_path + 'processed/07_train_df.parquet')

In [14]:
from sklearn.preprocessing import LabelEncoder

feature_cols = [col for col in train_df.columns if col not in ["ID", "Segment"]]

X = train_df[feature_cols].copy()
y = train_df["Segment"].copy()

# 타깃 라벨 인코딩
le_target = LabelEncoder()
y_encoded = le_target.fit_transform(y)

In [4]:
del train_df
gc.collect()

87

In [9]:
import numpy as np

In [15]:
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

In [16]:
X_test = test_df.copy()

encoders = {}  # 각 컬럼별 encoder 저장

for col in categorical_features:
    le_train = LabelEncoder()
    X[col] = le_train.fit_transform(X[col])
    encoders[col] = le_train
    unseen_labels_val = set(X_test[col]) - set(le_train.classes_)
    if unseen_labels_val:
        le_train.classes_ = np.append(le_train.classes_, list(unseen_labels_val))
    X_test[col] = le_train.transform(X_test[col])

In [9]:
X['Segment'] = y_encoded
X.to_parquet(data_path + 'processed/07_train_encoded.parquet')

In [2]:
import xgboost as xgb

In [3]:
df1 = pd.read_parquet(data_path + 'processed/07_train_encoded.parquet')
X = df1.drop('Segment', axis=1)
y_encoded = df1['Segment']
del df1
gc.collect()

6

In [17]:
try:
    model = xgb.XGBClassifier(
        tree_method='gpu_hist',
        gpu_id=0,
        random_state=42
    )
    model.fit(X, y_encoded)
except Exception:
    model = xgb.XGBClassifier(
        random_state=42
    )
    model.fit(X, y_encoded)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [01:24:09] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


In [18]:
X_test.drop(columns=['ID'],inplace=True)
# row-level 예측 수행
y_test_pred = model.predict(X_test)
# 예측 결과를 변환
y_test_pred_labels = le_target.inverse_transform(y_test_pred)

# row 단위 예측 결과를 test_data에 추가
test_data = test_df.copy()  # 원본 유지
test_data["pred_label"] = y_test_pred_labels

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [01:24:56] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [01:24:56] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


In [ ]:
from sklearn.metrics import f1_score
